In [1]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import numpy as np
import pandas as pd
from qdrant_client.models import PointStruct
from qdrant_client.models import Filter, FieldCondition, MatchValue
import os

# data

In [2]:
df = pd.read_json(r"E:\SUDO_FINAL\RAG2\app\data\local_description_final.jsonl", lines=True)
df["region"] = df["id"].str.extract(r'([A-Z])$')
region_map = {"B": "miền Bắc", "T": "miền Trung", "N": "miền Nam"}
df["region"] = df["region"].map(region_map)
df["region"] = df["region"].where(df["region"].notna(), None)
df

,id,word,video_url,description,region
0,D0005T,Miến Điện (nước Mi-an-ma),https://storage.googleapis.com/vsl-data-bucket...,"Tay phải giống số tự nhiên ""1"", lòng bàn tay h...",miền Trung
1,D0005B,Miến Điện (nước Mi-an-ma),https://storage.googleapis.com/vsl-data-bucket...,"Tay phải giống số tự nhiên ""1"", lòng bàn tay h...",miền Bắc
2,D0005N,Miến Điện (nước Mi-an-ma),https://storage.googleapis.com/vsl-data-bucket...,"Tay phải giống số tự nhiên ""1"", lòng bàn tay h...",miền Nam
3,D0001T,địa chỉ,https://storage.googleapis.com/vsl-data-bucket...,"1. Tay phải nắm, ngón trỏ duỗi thẳng, các ngón...",miền Trung
4,D0001B,địa chỉ,https://storage.googleapis.com/vsl-data-bucket...,"Tay phải nắm, ngón trỏ duỗi thẳng, các ngón cò...",miền Bắc
...,...,...,...,...,...
4357,W03938B,xanh xao,https://storage.googleapis.com/vsl-data-bucket...,"Tay phải mở, các ngón tay khép lại, lòng bàn t...",miền Bắc
4358,W03938N,xanh xao,https://storage.googleapis.com/vsl-data-bucket...,"Tay phải mở, các ngón tay khép lại, lòng bàn t...",miền Nam
4359,W03967N,xe khách,https://storage.googleapis.com/vsl-data-bucket...,"1. Hai tay nắm, ngón cái và ngón trỏ mở ra, lò...",miền Nam
4360,W03978,xếp,https://storage.googleapis.com/vsl-data-bucket...,"Hai tay khép, lòng bàn tay hướng vào nhau. Tay...",None


In [3]:
def build_full_description(row):
    desc = row["description"] if pd.notna(row["description"]) else ""
    region = f" ({row['region']})" if pd.notna(row["region"]) else ""
    return f"Ký hiệu của từ '{row['word']}'{region}: {desc}"

df["full_description"] = df.apply(build_full_description, axis=1)
df

,id,word,video_url,description,region,full_description
0,D0005T,Miến Điện (nước Mi-an-ma),https://storage.googleapis.com/vsl-data-bucket...,"Tay phải giống số tự nhiên ""1"", lòng bàn tay h...",miền Trung,Ký hiệu của từ 'Miến Điện (nước Mi-an-ma)' (mi...
1,D0005B,Miến Điện (nước Mi-an-ma),https://storage.googleapis.com/vsl-data-bucket...,"Tay phải giống số tự nhiên ""1"", lòng bàn tay h...",miền Bắc,Ký hiệu của từ 'Miến Điện (nước Mi-an-ma)' (mi...
2,D0005N,Miến Điện (nước Mi-an-ma),https://storage.googleapis.com/vsl-data-bucket...,"Tay phải giống số tự nhiên ""1"", lòng bàn tay h...",miền Nam,Ký hiệu của từ 'Miến Điện (nước Mi-an-ma)' (mi...
3,D0001T,địa chỉ,https://storage.googleapis.com/vsl-data-bucket...,"1. Tay phải nắm, ngón trỏ duỗi thẳng, các ngón...",miền Trung,Ký hiệu của từ 'địa chỉ' (miền Trung): 1. Tay ...
4,D0001B,địa chỉ,https://storage.googleapis.com/vsl-data-bucket...,"Tay phải nắm, ngón trỏ duỗi thẳng, các ngón cò...",miền Bắc,Ký hiệu của từ 'địa chỉ' (miền Bắc): Tay phải ...
...,...,...,...,...,...,...
4357,W03938B,xanh xao,https://storage.googleapis.com/vsl-data-bucket...,"Tay phải mở, các ngón tay khép lại, lòng bàn t...",miền Bắc,Ký hiệu của từ 'xanh xao' (miền Bắc): Tay phải...
4358,W03938N,xanh xao,https://storage.googleapis.com/vsl-data-bucket...,"Tay phải mở, các ngón tay khép lại, lòng bàn t...",miền Nam,Ký hiệu của từ 'xanh xao' (miền Nam): Tay phải...
4359,W03967N,xe khách,https://storage.googleapis.com/vsl-data-bucket...,"1. Hai tay nắm, ngón cái và ngón trỏ mở ra, lò...",miền Nam,Ký hiệu của từ 'xe khách' (miền Nam): 1. Hai t...
4360,W03978,xếp,https://storage.googleapis.com/vsl-data-bucket...,"Hai tay khép, lòng bàn tay hướng vào nhau. Tay...",None,"Ký hiệu của từ 'xếp': Hai tay khép, lòng bàn t..."


In [4]:
counts = df["region"].value_counts()
print(counts)

region
miền Nam      547
miền Bắc      534
miền Trung    522
Name: count, dtype: int64


In [5]:
texts = df["full_description"].tolist()
texts[:5]

['Ký hiệu của từ \'Miến Điện (nước Mi-an-ma)\' (miền Trung): Tay phải giống số tự nhiên "1", lòng bàn tay hướng sang trái, đầu ngón tay hướng lên trên. Đặt đầu ngón trỏ chạm vào sống mũi, sau đó đưa ngón trỏ sang phải, kéo dài ra ngoài.',
 'Ký hiệu của từ \'Miến Điện (nước Mi-an-ma)\' (miền Bắc): Tay phải giống số tự nhiên "1", lòng bàn tay hướng sang trái, đầu ngón tay hướng lên trên. Đặt đầu ngón trỏ chạm vào sống mũi, sau đó đưa ngón trỏ sang phải, kéo dài ra ngoài.',
 'Ký hiệu của từ \'Miến Điện (nước Mi-an-ma)\' (miền Nam): Tay phải giống số tự nhiên "1", lòng bàn tay hướng sang trái, đầu ngón tay hướng lên trên. Đặt đầu ngón trỏ chạm vào sống mũi, sau đó đưa ngón trỏ sang phải, kéo dài ra ngoài.',
 "Ký hiệu của từ 'địa chỉ' (miền Trung): 1. Tay phải nắm, ngón trỏ duỗi thẳng, các ngón khác gập lại, lòng bàn tay hướng xuống, đầu ngón tay hướng về phía trước, đưa tay xuống dưới một chút. 2. Giữ nguyên vị trí tay, lắc nhẹ cổ tay qua - lại 2 - 3 lần.",
 "Ký hiệu của từ 'địa chỉ' (miền

# embedding

## Dense embeddings

In [4]:
# Dense model
dense_model = SentenceTransformer("BAAI/bge-m3")
print(dense_model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False, 'architecture': 'XLMRobertaModel'})
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


## dense_embedding

In [5]:

dense_embeddings = dense_model.encode(
    df["word"].tolist(),
    convert_to_numpy=True,
    show_progress_bar=True
).astype("float32")

print("Embedding shape:", dense_embeddings.shape)


np.save("word_dense_embeddings.npy", dense_embeddings)

Batches:   0%|          | 0/137 [00:00<?, ?it/s]

Embedding shape: (4362, 1024)


In [ ]:
dense_embeddings = np.load("embeddings.npy")
dense_embeddings.shape

(4362, 1024)

# qdrant

In [6]:
#client_qdrant = QdrantClient(":memory:")
#client_qdrant = QdrantClient(path=r"E:\SUDO_FINAL\RAG2\app\data\qdrant_storage")    
client_qdrant = QdrantClient(path="word_embedding_qdrant_storage")    

In [7]:
client_qdrant.recreate_collection(
    collection_name="sign_vectors",
    vectors_config=VectorParams(
        size=dense_embeddings.shape[1],  
        distance=Distance.COSINE  
    )
)

C:\Users\ThienLaptop\AppData\Local\Temp\ipykernel_2380\2036446023.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client_qdrant.recreate_collection(


True

In [8]:
points = [
    PointStruct(
        id=i,
        vector=dense_embeddings[i],
        payload={
            "word": df.iloc[i]["word"],
            "region": df.iloc[i]["region"],
            "video": df.iloc[i]["video_url"],
            "description": df.iloc[i]["description"],
            "full_description": df.iloc[i]["full_description"]
        },
    )
    for i in range(len(df))
]

client_qdrant.upsert(collection_name="sign_vectors", points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

# query

## Only Dense Search

In [11]:
query = "trái cây"
query_vec = dense_model.encode([query], convert_to_numpy=True).astype("float32")[0]

results = client_qdrant.search(
    collection_name="sign_vectors",
    query_vector=query_vec,
    limit=10
)


for r in results:
    print(r)

id=1255 version=0 score=0.8439272046089172 payload={'word': 'cây ăn quả', 'region': 'miền Nam', 'video': 'https://storage.googleapis.com/vsl-data-bucket/videos/Videos/W00456N.mp4', 'description': '1. Tay trái mở, lòng bàn tay úp, các ngón tay khép, hướng về phía trước. 2. Tay phải mở, các ngón tay xoè, lòng bàn tay hướng sang trái, đặt lên mu bàn tay trái, đưa tay phải lên trên đồng thời các ngón tay phải rung nhẹ. 3. Tay phải chụm lại, lòng bàn tay hướng sang trái, đặt sát các đầu ngón tay vào khoé miệng phải, giữ nguyên cánh tay, lắc cổ tay qua - lại 2 - 3 lần.', 'full_description': "Ký hiệu của từ 'cây ăn quả' (miền Nam): 1. Tay trái mở, lòng bàn tay úp, các ngón tay khép, hướng về phía trước. 2. Tay phải mở, các ngón tay xoè, lòng bàn tay hướng sang trái, đặt lên mu bàn tay trái, đưa tay phải lên trên đồng thời các ngón tay phải rung nhẹ. 3. Tay phải chụm lại, lòng bàn tay hướng sang trái, đặt sát các đầu ngón tay vào khoé miệng phải, giữ nguyên cánh tay, lắc cổ tay qua - lại 2 - 3

C:\Users\ThienLaptop\AppData\Local\Temp\ipykernel_2380\1286694807.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client_qdrant.search(


In [12]:
query = "tờ báo"
query_vec = dense_model.encode([query], convert_to_numpy=True).astype("float32")[0]

region_filter = Filter(
    must=[
        FieldCondition(
            key="region",
            match=MatchValue(value="miền Trung")
        )
    ]
)

results = client_qdrant.search(
    collection_name="sign_vectors",
    query_vector=query_vec,
    query_filter=region_filter,  
    limit=10
)

for r in results:
    print(r)


C:\Users\ThienLaptop\AppData\Local\Temp\ipykernel_2380\2318023939.py:13: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client_qdrant.search(


id=3924 version=0 score=1.0 payload={'word': 'tờ báo', 'region': 'miền Trung', 'video': 'https://storage.googleapis.com/vsl-data-bucket/videos/Videos/W03508T.mp4', 'description': 'Hai tay khép, lòng bàn tay hướng vào nhau, các ngón tay duỗi thẳng, đầu ngón tay hướng về phía trước, đặt trước ngực. Sau đó, đồng thời mở hai tay sang hai bên, lòng bàn tay hướng ra ngoài.', 'full_description': "Ký hiệu của từ 'tờ báo' (miền Trung): Hai tay khép, lòng bàn tay hướng vào nhau, các ngón tay duỗi thẳng, đầu ngón tay hướng về phía trước, đặt trước ngực. Sau đó, đồng thời mở hai tay sang hai bên, lòng bàn tay hướng ra ngoài."} vector=None shard_key=None order_value=None
id=3942 version=0 score=0.7516928911209106 payload={'word': 'tờ giấy', 'region': 'miền Trung', 'video': 'https://storage.googleapis.com/vsl-data-bucket/videos/Videos/W03509T.mp4', 'description': 'Tay trái khép, lòng bàn tay hướng lên trên, đầu ngón tay hướng về phía trước, đặt ngang tầm ngực. Tay phải nắm, ngón cái và ngón trỏ mở r

In [13]:
query = "nhà hàng"
query_vec = dense_model.encode([query], convert_to_numpy=True).astype("float32")[0]

region_filter = Filter(
    must=[
        FieldCondition(
            key="region",
            match=MatchValue(value="miền Trung")
        )
    ]
)

results = client_qdrant.search(
    collection_name="sign_vectors",
    query_vector=query_vec,
    query_filter=region_filter,  
    limit=10
)

for r in results:
    print(r)


id=1754 version=0 score=0.720878005027771 payload={'word': 'cửa hàng', 'region': 'miền Trung', 'video': 'https://storage.googleapis.com/vsl-data-bucket/videos/Videos/W00935T.mp4', 'description': 'Hai tay khép, lòng bàn tay hướng vào nhau, đầu ngón tay hướng lên trên, đặt song song trước mặt. Giữ nguyên tay trái, đưa tay phải sang phải đồng thời xoay cổ tay phải sang phải.', 'full_description': "Ký hiệu của từ 'cửa hàng' (miền Trung): Hai tay khép, lòng bàn tay hướng vào nhau, đầu ngón tay hướng lên trên, đặt song song trước mặt. Giữ nguyên tay trái, đưa tay phải sang phải đồng thời xoay cổ tay phải sang phải."} vector=None shard_key=None order_value=None
id=872 version=0 score=0.6591209173202515 payload={'word': 'bán hàng', 'region': 'miền Trung', 'video': 'https://storage.googleapis.com/vsl-data-bucket/videos/Videos/W00107T.mp4', 'description': 'Tay phải mở, lòng bàn tay hướng lên trên, đặt trước mặt, đưa tay ra phía trước và hơi xuống dưới, đồng thời gập nhẹ các ngón tay.', 'full_des

C:\Users\ThienLaptop\AppData\Local\Temp\ipykernel_2380\2687051046.py:13: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client_qdrant.search(


# LLM 

In [14]:
from google import genai
import os
from dotenv import load_dotenv

In [15]:
# Load .env
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

In [16]:
import json

client_llm = genai.Client(api_key=api_key)

user_query = "Tôi chuẩn bị đi ăn tối ở nhà hàng với bạn bè vào cuối tuần này. Bạn có thể giúp tôi học một số từ ngữ ký hiệu liên quan đến việc đi ăn không?"

intent_prompt = f"""
Phân loại truy vấn sau thành một trong 3 loại:
1. context_search: người dùng nói về tình huống, ngữ cảnh (ví dụ đi ăn, đi học, đi chơi,...)
2. word_learning: người dùng muốn học một từ cụ thể trong ngôn ngữ ký hiệu
3. action_description: người dùng mô tả động tác (tay, cánh tay, bàn tay, đầu, nắm, chỉ,...)

Truy vấn: {user_query}

Hãy trả về ở dạng sau:
Intent: <intent>
Keywords: [<keyword1>, <keyword2>, <keyword3>, ...]

Trong đó:
Keywords: từ khóa chính hoặc cụm từ quan trọng có liên quan đến câu hỏi của người dùng.
"""

resp = client_llm.models.generate_content(
    model="gemini-2.0-flash",
    contents=intent_prompt
)

resp.text

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


'Intent: context_search\nKeywords: ["đi ăn tối", "nhà hàng", "bạn bè", "cuối tuần", "ngôn ngữ ký hiệu", "đi ăn"]\n'

In [18]:
import re

# get intent
intent_match = re.search(r"Intent:\s*(\w+)", resp.text)
intent = intent_match.group(1) if intent_match else ""



# get list keywords
keywords_match = re.search(r"Keywords:\s*\[(.*?)\]", resp.text, re.S)
if keywords_match:
    keywords_raw = keywords_match.group(1)
    keywords = [kw.strip().strip('"').strip("'") for kw in keywords_raw.split(",")]
else:
    keywords = []

print("Intent:", intent)
print("Keywords:", keywords)

Intent: context_search
Keywords: ['đi ăn tối', 'nhà hàng', 'bạn bè', 'cuối tuần', 'ngôn ngữ ký hiệu', 'đi ăn']


In [19]:
def search_sign_videos(keywords, top_k=10):
    all_results = []
    seen_ids = set() 

    for kw in keywords:
        print(f"🔍 Searching for keyword: {kw}")
        query_vec = dense_model.encode([kw], convert_to_numpy=True).astype("float32")[0]

        results = client_qdrant.search(
            collection_name="sign_vectors",
            query_vector=query_vec,
            limit=top_k
        )

        for r in results:
            if r.id not in seen_ids:
                seen_ids.add(r.id)
                all_results.append({
                    "score": float(r.score),
                    "word": r.payload["word"],
                    "region": r.payload["region"],
                    "video": r.payload["video"],
                    "description": r.payload["description"],
                })
    return all_results

In [20]:
if intent in ["context_search", "word_learning", "action_description"]:
    results = search_sign_videos(keywords, top_k=10)

    for r in results:
        print(r)

🔍 Searching for keyword: đi ăn tối


C:\Users\ThienLaptop\AppData\Local\Temp\ipykernel_2380\875562501.py:9: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client_qdrant.search(


🔍 Searching for keyword: nhà hàng
🔍 Searching for keyword: bạn bè
🔍 Searching for keyword: cuối tuần
🔍 Searching for keyword: ngôn ngữ ký hiệu
🔍 Searching for keyword: đi ăn
{'score': 0.8124939203262329, 'word': 'buổi tối', 'region': 'miền Bắc', 'video': 'https://storage.googleapis.com/vsl-data-bucket/videos/Videos/W00369B.mp4', 'description': 'Hai tay khép, lòng bàn tay hướng vào trong, đầu ngón tay hướng lên trên, đặt song song trước mặt, đưa đồng thời hai tay lại gần và bắt chéo nhau (tay phải đặt lên tay trái), lòng bàn tay hướng xuống dưới.'}
{'score': 0.8124939203262329, 'word': 'buổi tối', 'region': 'miền Trung', 'video': 'https://storage.googleapis.com/vsl-data-bucket/videos/Videos/W00369T.mp4', 'description': 'Hai tay khép, lòng bàn tay hướng vào trong, đầu ngón tay hướng lên trên, đặt song song trước mặt, đưa đồng thời hai tay lại gần và bắt chéo nhau (tay phải đặt lên tay trái), lòng bàn tay hướng xuống dưới.'}
{'score': 0.8124939203262329, 'word': 'buổi tối', 'region': 'miề